In [1]:
import numpy as np
import pandas as pd
import pickle


# EEGNet-specific imports
from EEGModels import ShallowConvNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.viz import plot_confusion_matrix

# # sklearn imports
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report, accuracy_score, f1_score, balanced_accuracy_score

# # tools for plotting confusion matrices
# import matplotlib
# from matplotlib import pyplot as plt
# import seaborn as sn

In [2]:
def load_dataset_ctrvsone_and_transfer(user_split = True, runidx=1, data_type='meditation', one_class='HT'):
    X_train = None
    y_train_medid = None
    y_train_subjid = None
    X_val = None
    y_val_medid = None
    y_val_subjid = None
    X_test = None
    y_test_medid = None
    y_test_subjid = None

    dtype_names = ["HT", "SNY", "VIP", "CTR"]

    # user_split: Determines the creation of train/val/test set. In user_split, test/val users are never seen during the train. time_split randomly splits each user's chunks into train/val/test.
    if user_split:
        data_file_path = '../../iconip_data/{}/user_based_splits_with_timestamp_RUN{}.pkl'.format(data_type, runidx)
    else:
        data_file_path = '../../iconip_data/{}/time_based_splits_with_timestamp_RUN{}.pkl'.format(data_type, runidx)

    with open(data_file_path, 'rb') as f:
        all_data_splits = pickle.load(f)

        data_index_label_oneclass = dtype_names.index(one_class)
        data_index_label_ctr = dtype_names.index('CTR')
        transfer_index_labels = [x for x in list(set([0,1,2,3]).difference(set([dtype_names.index(one_class), 3]))) ]
        
        X_train = all_data_splits['train']['x']
        y_train_medid = all_data_splits['train']['y_med']
        y_train_subjid = all_data_splits['train']['y_subj']
        y_train_ts = all_data_splits['train']['y_ts']
        clf_config_subset_indices = np.where((y_train_medid == data_index_label_oneclass) | (y_train_medid == data_index_label_ctr))[0]
        X_train = X_train[(clf_config_subset_indices)]
        y_train_medid = y_train_medid[(clf_config_subset_indices)]
        y_train_subjid = y_train_subjid[clf_config_subset_indices]
        y_train_ts = y_train_ts[(clf_config_subset_indices)]

        X_val = all_data_splits['val']['x']
        y_val_medid = all_data_splits['val']['y_med']
        y_val_subjid = all_data_splits['val']['y_subj']
        y_val_ts = all_data_splits['val']['y_ts']
        clf_config_subset_indices = np.where((y_val_medid == data_index_label_oneclass) | (y_val_medid == data_index_label_ctr))[0]
        X_val = X_val[(clf_config_subset_indices)]
        y_val_medid = y_val_medid[(clf_config_subset_indices)]
        y_val_subjid = y_val_subjid[(clf_config_subset_indices)]
        y_val_ts = y_val_ts[(clf_config_subset_indices)]
        
        # Pretrain test and transfer test set
        X_test = all_data_splits['test']['x']
        y_test_medid = all_data_splits['test']['y_med']
        y_test_subjid = all_data_splits['test']['y_subj']
        y_test_ts = all_data_splits['test']['y_ts']
        
        # transfer test contains CTR from test and other two classes from the test.
        ctr_othertwo_indices = np.where((y_test_medid == data_index_label_ctr) | (y_test_medid == transfer_index_labels[0]) | (y_test_medid == transfer_index_labels[1]))[0]
        X_test_transfer = X_test[(ctr_othertwo_indices)]
        y_test_transfer_medid = y_test_medid[(ctr_othertwo_indices)]
        y_test_transfer_subjid = y_test_subjid[(ctr_othertwo_indices)]
        
        # pretrain test filter CTR and one_class from the test.
        clf_config_subset_indices = np.where((y_test_medid == data_index_label_oneclass) | (y_test_medid == data_index_label_ctr))[0]
        X_test = X_test[(clf_config_subset_indices)]
        y_test_medid = y_test_medid[(clf_config_subset_indices)]
        y_test_subjid = y_test_subjid[(clf_config_subset_indices)]
        y_test_ts = y_test_ts[(clf_config_subset_indices)]

    return X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, X_test, y_test_medid, y_test_subjid, y_test_ts, X_test_transfer, y_test_transfer_medid, y_test_transfer_subjid

In [3]:
def train_model(user_split, med_tech_clf, data_type, one_class):
    
    kernels, chans, samples = 1, 64, 2560
    
    split_strat = 'user' if user_split else 'time'
    model_results = pd.DataFrame()

    for runidx in range(1, 6):
        model_results['{}_Train_Acc'.format(runidx)] = None
        model_results['{}_Val_Acc'.format(runidx)] = None
        model_results['{}_Test_Acc'.format(runidx)] = None
        model_results['{}_Test_Transfer_Acc'.format(runidx)] = None
        model_results['{}_best_params'.format(runidx)] = None


    for runidx in range(1, 6):
        print("RUN ID: ", runidx)
        med_tech_clf = True
        
        X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, X_test, y_test_medid, y_test_subjid, y_test_ts, X_transfer, y_test_transfer_medid, y_test_transfer_subjid = load_dataset_ctrvsone_and_transfer(user_split=user_split, runidx=runidx, data_type=data_type, one_class=one_class)
        ## Convert data to binary classification task, i.e. meditation expert as class 0 vs control group as class 1
        y_train_medid = y_train_medid//3
        y_val_medid = y_val_medid//3
        y_test_medid = y_test_medid//3
        y_test_transfer_medid = y_test_transfer_medid//3

        best_clf_ours = None
        best_clf_val = 0

        #X_test_transfer, y_test_transfer_medid, y_test_transfer_subjid
        if med_tech_clf:
            Y_train      = y_train_medid
            Y_validate   = y_val_medid
            Y_test       = y_test_medid
            Y_transfer   = y_test_transfer_medid
            ckpt_file_suffix = '{}_med_clf'.format(split_strat)
        else:
            Y_train      = y_train_subjid
            Y_validate   = y_val_subjid
            Y_test       = y_test_subjid
            Y_transfer   = y_test_transfer_subjid
            ckpt_file_suffix = '{}_subj_clf'.format(split_strat)
        nb_classes = np.unique(Y_train).shape[0]
        print(np.unique(Y_train), ": unique class labels.")
    
        ############################# EEGNet portion ##################################
        # convert labels to one-hot encodings.
        Y_train      = np_utils.to_categorical(Y_train)
        Y_validate   = np_utils.to_categorical(Y_validate)
        Y_test       = np_utils.to_categorical(Y_test)
        Y_transfer   = np_utils.to_categorical(Y_transfer)

        # convert data to NHWC (trials, channels, samples, kernels) format. Data 
        # contains 60 channels and 151 time-points. Set the number of kernels to 1.
        X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
        X_val        = X_val.reshape(X_val.shape[0], chans, samples, kernels)
        X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
        X_transfer   = X_transfer.reshape(X_transfer.shape[0], chans, samples, kernels)

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_test.shape[0], 'test samples')
        print(X_transfer.shape[0], 'transfer samples')
        print(nb_classes, " number of classes")
    
        # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
        # model configurations may do better, but this is a good starting point)
        model = ShallowConvNet(nb_classes = nb_classes, Chans = chans, Samples = samples, 
                    dropoutRate = 0.5)

        # compile the model and set the optimizers
        model.compile(loss='categorical_crossentropy', optimizer='adam', 
                    metrics = ['accuracy'])

        # count number of parameters in the model
        numParams    = model.count_params()    

        # set a valid path for your system to record model checkpoints
        checkpointer = ModelCheckpoint(filepath='./ckpt/shallowconv/checkpoint_{}.h5'.format(ckpt_file_suffix), 
                                    verbose=1, save_best_only=True)
        
        ###############################################################################
        # if the classification task was imbalanced (significantly more trials in one
        # class versus the others) you can assign a weight to each class during 
        # optimization to balance it out. This data is approximately balanced so we 
        # don't need to do this, but is shown here for illustration/completeness. 
        ###############################################################################

        # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
        # the weights all to be 1
        class_weights = {cl: 1 for cl in range(nb_classes)}

        ################################################################################
        # fit the model. Due to very small sample sizes this can get
        # pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
        # Riemannian geometry classification (below)
        ################################################################################
        escallback = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)
        fittedModel = model.fit(X_train, Y_train, batch_size = 128, epochs = 30, 
                                verbose = 2, validation_data=(X_val, Y_validate),
                                callbacks=[checkpointer, escallback], class_weight = class_weights)

        # load optimal weights
        model.load_weights('./ckpt/shallowconv/checkpoint_{}.h5'.format(ckpt_file_suffix))
    
        probs       = model.predict(X_test)
        preds       = probs.argmax(axis = -1)

        probs_transfer = model.predict(X_transfer)
        preds_transfer = probs_transfer.argmax(axis = -1)
        
        # save preds on train and val set for error analysis
        preds_train_eegnet = model.predict(X_train)
        preds_train_eegnet = preds_train_eegnet.argmax(axis = -1)
        preds_val_eegnet = model.predict(X_val)
        preds_val_eegnet = preds_val_eegnet.argmax(axis = -1)
        
        
        acc_test         = np.mean(preds == Y_test.argmax(axis=-1))
        acc_transfer     = np.mean(preds_transfer == Y_transfer.argmax(axis=-1))
        print("ShallowConv Classification accuracy test and transfer: %f %f" % (acc_test, acc_transfer))
        model_results.loc["ShallowConv", ['{}_Train_Acc'.format(runidx), '{}_Val_Acc'.format(runidx), '{}_Test_Acc'.format(runidx), '{}_Test_Transfer_Acc'.format(runidx)]] = [preds_train_eegnet, preds_val_eegnet, acc_test, acc_transfer]
    if user_split:
        model_results.to_csv('./RESULTS_T5{}_MED_TransferPRED_{}_CTRvs{}_ShallowConv.csv'.format('USER', data_type, one_class))
    else:
        model_results.to_csv('./RESULTS_T5_MED_TransferPRED_{}_CTRvs{}_ShallowConv.csv'.format(data_type, one_class))

    return model_results

In [4]:
# time split
for one_class in ['HT', 'SNY', 'VIP']:
    print(one_class)
    model_results = train_model(user_split=False, med_tech_clf=True, data_type='meditation', one_class=one_class)

HT
RUN ID:  1
[0 1] : unique class labels.
X_train shape: (990, 64, 2560, 1)
990 train samples
344 test samples
523 transfer samples
2  number of classes


2022-06-23 13:20:13.015559: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/30


2022-06-23 13:20:16.308139: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3339714560 exceeds 10% of free system memory.
2022-06-23 13:20:17.472934: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3339714560 exceeds 10% of free system memory.
2022-06-23 13:20:28.568149: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3339714560 exceeds 10% of free system memory.
2022-06-23 13:20:29.522891: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3339714560 exceeds 10% of free system memory.
2022-06-23 13:20:37.122904: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 3339714560 exceeds 10% of free system memory.



Epoch 1: val_loss improved from inf to 2.79276, saving model to ./ckpt/shallowconv/checkpoint_time_med_clf.h5
8/8 - 77s - loss: 2.6269 - accuracy: 0.5465 - val_loss: 2.7928 - val_accuracy: 0.5015 - 77s/epoch - 10s/step
Epoch 2/30

Epoch 2: val_loss did not improve from 2.79276
8/8 - 73s - loss: 1.6995 - accuracy: 0.6101 - val_loss: 3.2862 - val_accuracy: 0.5740 - 73s/epoch - 9s/step
Epoch 3/30

Epoch 3: val_loss improved from 2.79276 to 0.81229, saving model to ./ckpt/shallowconv/checkpoint_time_med_clf.h5
8/8 - 75s - loss: 1.4789 - accuracy: 0.6677 - val_loss: 0.8123 - val_accuracy: 0.7372 - 75s/epoch - 9s/step
Epoch 4/30

Epoch 4: val_loss improved from 0.81229 to 0.47031, saving model to ./ckpt/shallowconv/checkpoint_time_med_clf.h5
8/8 - 70s - loss: 0.6310 - accuracy: 0.8030 - val_loss: 0.4703 - val_accuracy: 0.7583 - 70s/epoch - 9s/step
Epoch 5/30

Epoch 5: val_loss did not improve from 0.47031
8/8 - 68s - loss: 0.6513 - accuracy: 0.7606 - val_loss: 0.6631 - val_accuracy: 0.7100 

...

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (996, 64, 2560, 1)
996 train samples
351 test samples
516 transfer samples
2  number of classes
Epoch 1/30

Epoch 1: val_loss improved from inf to 1.26964, saving model to ./ckpt/shallowconv/checkpoint_time_med_clf.h5
8/8 - 75s - loss: 2.5698 - accuracy: 0.6225 - val_loss: 1.2696 - val_accuracy: 0.4337 - 75s/epoch - 9s/step
Epoch 2/30

Epoch 2: val_loss did not improve from 1.26964
8/8 - 79s - loss: 1.4373 - accuracy: 0.6074 - val_loss: 1.3180 - val_accuracy: 0.6687 - 79s/epoch - 10s/step
Epoch 3/30

Epoch 3: val_loss did not improve from 1.26964
8/8 - 72s - loss: 0.8716 - accuracy: 0.7159 - val_loss: 2.0338 - val_accuracy: 0.6837 - 72s/epoch - 9s/step
Epoch 4/30

Epoch 4: val_loss improved from 1.26964 to 0.54757, saving model to ./ckpt/shallowconv/checkpoint_time_med_clf.h5
8/8 - 71s - loss: 1.6332 - accuracy: 0.6908 - val_loss: 0.5476 - val_accuracy: 0.7831 - 71s/epoch - 9s/step
Epoch 5/30

Epoch 5: val_loss improved from 0.54757 to 0.4051

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (996, 64, 2560, 1)
996 train samples
351 test samples
516 transfer samples
2  number of classes
Epoch 1/30

Epoch 1: val_loss improved from inf to 1.76496, saving model to ./ckpt/shallowconv/checkpoint_time_med_clf.h5
8/8 - 68s - loss: 1.9905 - accuracy: 0.5683 - val_loss: 1.7650 - val_accuracy: 0.6687 - 68s/epoch - 8s/step
Epoch 2/30

Epoch 2: val_loss improved from 1.76496 to 0.55722, saving model to ./ckpt/shallowconv/checkpoint_time_med_clf.h5
8/8 - 71s - loss: 1.1578 - accuracy: 0.6847 - val_loss: 0.5572 - val_accuracy: 0.7560 - 71s/epoch - 9s/step
Epoch 3/30

Epoch 3: val_loss improved from 0.55722 to 0.50244, saving model to ./ckpt/shallowconv/checkpoint_time_med_clf.h5
8/8 - 76s - loss: 0.5506 - accuracy: 0.7811 - val_loss: 0.5024 - val_accuracy: 0.7500 - 76s/epoch - 9s/step
Epoch 4/30

Epoch 4: val_loss improved from 0.50244 to 0.37344, saving model to ./ckpt/shallowconv/checkpoint_time_med_clf.h5
8/8 - 76s - loss: 0.4264 - accuracy:

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [5]:
# User split
for one_class in ['HT', 'SNY', 'VIP']:
    print(one_class)
    model_results = train_model(user_split=True, med_tech_clf=True, data_type='meditation', one_class=one_class)

HT
RUN ID:  1
[0 1] : unique class labels.
X_train shape: (1038, 64, 2560, 1)
1038 train samples
308 test samples
447 transfer samples
2  number of classes
Epoch 1/30

Epoch 1: val_loss improved from inf to 1.54495, saving model to ./ckpt/shallowconv/checkpoint_user_med_clf.h5
9/9 - 74s - loss: 1.4714 - accuracy: 0.5645 - val_loss: 1.5449 - val_accuracy: 0.6458 - 74s/epoch - 8s/step
Epoch 2/30

Epoch 2: val_loss did not improve from 1.54495
9/9 - 76s - loss: 0.4655 - accuracy: 0.8218 - val_loss: 1.6143 - val_accuracy: 0.6082 - 76s/epoch - 8s/step
Epoch 3/30

Epoch 3: val_loss did not improve from 1.54495
9/9 - 80s - loss: 0.9319 - accuracy: 0.8863 - val_loss: 3.6409 - val_accuracy: 0.6897 - 80s/epoch - 9s/step
Epoch 4/30

Epoch 4: val_loss did not improve from 1.54495
9/9 - 83s - loss: 0.4679 - accuracy: 0.8410 - val_loss: 2.5286 - val_accuracy: 0.6677 - 83s/epoch - 9s/step
10/10 [==============================] - 3s 301ms/step
ShallowConv Classification accuracy test and transfer: 0.3

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


[0 1] : unique class labels.
X_train shape: (1031, 64, 2560, 1)
1031 train samples
325 test samples
460 transfer samples
2  number of classes
Epoch 1/30

Epoch 1: val_loss improved from inf to 1.58087, saving model to ./ckpt/shallowconv/checkpoint_user_med_clf.h5
9/9 - 80s - loss: 4.1230 - accuracy: 0.4675 - val_loss: 1.5809 - val_accuracy: 0.4175 - 80s/epoch - 9s/step
Epoch 2/30

Epoch 4: val_loss did not improve from 1.06450
9/9 - 74s - loss: 0.7588 - accuracy: 0.7177 - val_loss: 1.4315 - val_accuracy: 0.4984 - 74s/epoch - 8s/step
Epoch 5/30

Epoch 5: val_loss did not improve from 1.06450
9/9 - 74s - loss: 0.5775 - accuracy: 0.7556 - val_loss: 1.3398 - val_accuracy: 0.4693 - 74s/epoch - 8s/step
Epoch 6/30

Epoch 6: val_loss did not improve from 1.06450
9/9 - 69s - loss: 0.4282 - accuracy: 0.8147 - val_loss: 1.1111 - val_accuracy: 0.6246 - 69s/epoch - 8s/step
10/10 [==============================] - 3s 274ms/step
ShallowConv Classification accuracy test and transfer: 0.652308 0.639130

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1058, 64, 2560, 1)
1058 train samples
311 test samples
485 transfer samples
2  number of classes
Epoch 1/30

Epoch 1: val_loss improved from inf to 3.47968, saving model to ./ckpt/shallowconv/checkpoint_user_med_clf.h5
9/9 - 71s - loss: 3.0280 - accuracy: 0.5624 - val_loss: 3.4797 - val_accuracy: 0.6385 - 71s/epoch - 8s/step
Epoch 2/30

Epoch 2: val_loss improved from 3.47968 to 0.53341, saving model to ./ckpt/shallowconv/checkpoint_user_med_clf.h5
9/9 - 68s - loss: 2.2782 - accuracy: 0.6853 - val_loss: 0.5334 - val_accuracy: 0.7500 - 68s/epoch - 8s/step
Epoch 3/30

Epoch 3: val_loss did not improve from 0.53341
9/9 - 68s - loss: 2.0497 - accuracy: 0.6654 - val_loss: 1.3628 - val_accuracy: 0.6892 - 68s/epoch - 8s/step
Epoch 4/30

Epoch 4: val_loss did not improve from 0.53341
9/9 - 69s - loss: 1.1537 - accuracy: 0.6815 - val_loss: 0.7978 - val_accuracy: 0.6554 - 69s/epoch - 8s/step
Epoch 5/30

Epoch 5: val_loss did not improve from 0.53341
9

...

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1042, 64, 2560, 1)
1042 train samples
315 test samples
461 transfer samples
2  number of classes
Epoch 1/30

Epoch 1: val_loss improved from inf to 0.87055, saving model to ./ckpt/shallowconv/checkpoint_user_med_clf.h5
9/9 - 58s - loss: 2.5242 - accuracy: 0.5461 - val_loss: 0.8705 - val_accuracy: 0.5714 - 58s/epoch - 6s/step
Epoch 2/30

Epoch 2: val_loss did not improve from 0.87055
9/9 - 57s - loss: 1.5647 - accuracy: 0.6900 - val_loss: 2.2311 - val_accuracy: 0.5497 - 57s/epoch - 6s/step
Epoch 3/30

Epoch 3: val_loss did not improve from 0.87055
9/9 - 55s - loss: 0.4337 - accuracy: 0.8349 - val_loss: 0.9109 - val_accuracy: 0.5590 - 55s/epoch - 6s/step
Epoch 4/30

Epoch 4: val_loss improved from 0.87055 to 0.73550, saving model to ./ckpt/shallowconv/checkpoint_user_med_clf.h5
9/9 - 57s - loss: 0.2355 - accuracy: 0.9060 - val_loss: 0.7355 - val_accuracy: 0.6957 - 57s/epoch - 6s/step
Epoch 5/30

Epoch 5: val_loss did not improve from 0.73550
9

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1045, 64, 2560, 1)
1045 train samples
316 test samples
486 transfer samples
2  number of classes
Epoch 1/30

Epoch 1: val_loss improved from inf to 0.75850, saving model to ./ckpt/shallowconv/checkpoint_user_med_clf.h5
9/9 - 55s - loss: 2.5147 - accuracy: 0.5474 - val_loss: 0.7585 - val_accuracy: 0.5094 - 55s/epoch - 6s/step
Epoch 2/30

Epoch 2: val_loss did not improve from 0.75850
9/9 - 61s - loss: 0.8990 - accuracy: 0.7158 - val_loss: 0.9517 - val_accuracy: 0.5252 - 61s/epoch - 7s/step
Epoch 3/30

Epoch 3: val_loss did not improve from 0.75850
9/9 - 59s - loss: 0.6065 - accuracy: 0.7215 - val_loss: 1.6420 - val_accuracy: 0.5377 - 59s/epoch - 7s/step
Epoch 4/30

Epoch 4: val_loss did not improve from 0.75850
9/9 - 60s - loss: 0.6162 - accuracy: 0.7684 - val_loss: 1.5567 - val_accuracy: 0.5220 - 60s/epoch - 7s/step
10/10 [==============================] - 3s 253ms/step
ShallowConv Classification accuracy test and transfer: 0.772152 0.547325

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [6]:
print("Done")

Done
